In [86]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import seaborn as sns


In [87]:
import numpy as np
import pandas as pd
from collections import defaultdict

In [88]:
import datetime as dt
from dateutil.relativedelta import relativedelta

In [89]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import desc

In [90]:
engine = create_engine("sqlite:///db/Veggie_Fruit_DB.sqlite")

In [91]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [92]:
# We can view all of the classes that automap found
Base.classes.keys()

['fruitexportdest',
 'fruitexportval',
 'fruitimportsource',
 'fruitimportval',
 'veggieexportdest',
 'veggieexportval',
 'veggieimportsource',
 'veggieimportval']

In [93]:
inspector=inspect(engine)
inspector.get_table_names()

['fruitexportdest',
 'fruitexportval',
 'fruitimportsource',
 'fruitimportval',
 'veggieexportdest',
 'veggieexportval',
 'veggieimportsource',
 'veggieimportval']

In [94]:
measurement_columns = inspector.get_columns('veggieexportdest')
for c in measurement_columns:
    print(c['name'])

id
product
flow
country
share
avg
yr15
yr16
yr17
yr18
yr19
lat
lon


In [95]:
# Save references to each table
# FED = FruistExportDestination, FEV =FruitExportValue, FIS =FruitImportSource,FIV=FruitImportValue
# VED =VeggieExportDestination, VEV=VeggieExportValue, VIS=VeggieImportSource, VIV=VeggieImportValue
FED = Base.classes.fruitexportdest
# FEV = Base.classes.fruitexportval
FIS = Base.classes.fruitimportsource
# FIV = Base.classes.fruitimportval
VED = Base.classes.veggieexportdest
# VEV = Base.classes.veggieexportval
VIS = Base.classes.veggieimportsource
# VIV = Base.classes.veggieimportval


In [96]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [132]:
sel = [VED.product, VED.country, VED.share]
market_year = session.query(*sel).statement
mkt_yr_df = pd.read_sql_query(market_year, con = engine)
product_df = mkt_yr_df.groupby(['product']).count()
product_list = product_df.index.values.tolist()
product_list

['artichoke',
 'beet',
 'broccoli',
 'carrot',
 'green pea',
 'mushroom',
 'onion',
 'pepper',
 'potato',
 'sweet corn']

In [131]:
sel = [FED.product, FED.country, FED.share]
fruit_summary = session.query(*sel).statement
fruit_summary_df = pd.read_sql_query(fruit_summary, con = engine)
fruit_df = fruit_summary_df.groupby(['product']).count()
fruit_list = fruit_df.index.values.tolist()
fruit_summary_df.head()

,product,country,share
0,Apple,Mexico,26.40%
1,Apple,Canada,17.10%
2,Apple,India,10.10%
3,Apple,Taiwan,8.10%
4,Apple,Vietnam,4.40%


In [129]:
sel = [VED.product, VED.country, VED.share, VED.lat, VED.lon]
results = session.query(*sel).filter(VED.product == 'potato').all()
veggie_list = []
for result in results:
    veggie_country = {}
    veggie_country["Country"] = result[1]
    veggie_country["Type"] = "veggie"
    veggie_country["Share"] = float(result[2].replace('%',''))
    veggie_country["lat"] = result[3]
    veggie_country["lon"] = result[4]
    veggie_list.append(veggie_country)

veggie_list

[{'Country': 'Canada',
  'Type': 'veggie',
  'Share': 46.1,
  'lat': 56.130366,
  'lon': -106.346771},
 {'Country': 'Mexico',
  'Type': 'veggie',
  'Share': 21.0,
  'lat': 23.634501,
  'lon': -102.552784},
 {'Country': 'Japan',
  'Type': 'veggie',
  'Share': 7.4,
  'lat': 36.204824,
  'lon': 138.252924},
 {'Country': 'Taiwan',
  'Type': 'veggie',
  'Share': 5.7,
  'lat': 23.69781,
  'lon': 120.960515},
 {'Country': 'Korea, South',
  'Type': 'veggie',
  'Share': 4.5,
  'lat': 35.907757,
  'lon': 127.766922}]

In [98]:
fe_summary = session.query(FED.country,FED.yr17,FED.yr18).group_by(FED.country).order_by(FED.yr18.desc()).all()
fe_summary_df = pd.DataFrame(fe_summary, columns=['country', 'yr17', 'yr18'])
fe_summary_df['yr17'] = fe_summary_df['yr17'].str.replace(',', '')
fe_summary_df['yr18'] = fe_summary_df['yr18'].str.replace(',', '')
fe_summary_df=fe_summary_df.dropna(how="any")
fe_summary_df["yr17"] = pd.to_numeric(fe_summary_df["yr17"])
fe_summary_df["yr18"] = pd.to_numeric(fe_summary_df["yr18"])

ve_summary = session.query(VED.country,VED.yr17,VED.yr18).group_by(VED.country).order_by(VED.yr18.desc()).all()
ve_summary_df = pd.DataFrame(ve_summary, columns=['country', 'yr17', 'yr18'])
ve_summary_df['yr17'] = ve_summary_df['yr17'].str.replace(',', '')
ve_summary_df['yr18'] = ve_summary_df['yr18'].str.replace(',', '')
ve_summary_df=ve_summary_df.dropna(how="any")
ve_summary_df["yr17"] = pd.to_numeric(ve_summary_df["yr17"])
ve_summary_df["yr18"] = pd.to_numeric(ve_summary_df["yr18"])

esummary_frames = [ve_summary_df, fe_summary_df]
summary_export = pd.concat(esummary_frames,ignore_index=False)
summary_export = summary_export.sort_values(['yr17', 'yr18'], ascending=False)
export_yr17sum=summary_export['yr17'].sum()
export_yr18sum=summary_export['yr18'].sum()


e_ms17=summary_export['yr17']/export_yr17sum*100
e_ms18=summary_export['yr18']/export_yr18sum*100
summary_export["MarketShare(2017)"]=e_ms17
summary_export["MarketShare(2018)"]=e_ms18
summary_export["ChangeInMarketShare"] = e_ms18-e_ms17
renamed_summary_export = summary_export.rename(columns={"country":"Country", "MS 2017":"MarketShare(2017)","MS 2018":"MarketShare(2018)", "Change in MS":"ChangeInMarketShare","yr17":"Value(2017)", "yr18":"Value(2018)"})
summary_export = renamed_summary_export[["Country","MarketShare(2017)","MarketShare(2018)","ChangeInMarketShare", "Value(2017)", "Value(2018)"]]
summary_export=summary_export.sort_values(['MarketShare(2018)'], ascending=False).head()
Summary_Export = summary_export[["Country", "MarketShare(2017)", "MarketShare(2018)", "ChangeInMarketShare", "Value(2017)", "Value(2018)"]]
summary_export_list = []

Summary_Export = Summary_Export.reset_index(drop=True)
ms17_list = Summary_Export["MarketShare(2017)"].tolist()
ms18_list = Summary_Export["MarketShare(2018)"].tolist()
mschange_list = Summary_Export["ChangeInMarketShare"].tolist()
value_yr17 = Summary_Export["Value(2017)"].tolist()
value_yr18 = Summary_Export["Value(2018)"].tolist()

country_list = Summary_Export["Country"].tolist()
for i in range(len(country_list)):
    summary_report_dict = {}
    summary_report_dict["Country"] = country_list[i]
    summary_report_dict["MS17"] = round(ms17_list[i],1)
    summary_report_dict["MS18"] = round(ms18_list[i],1)
    summary_report_dict["MSChange"] = round(mschange_list[i],1)
    summary_report_dict["Value_yr17"] = value_yr17[i]
    summary_report_dict["Value_yr18"] = value_yr18[i]
    
    summary_export_list.append(summary_report_dict)
summary_export_list

[{'Country': 'Mexico',
  'MS17': 25.7,
  'MS18': 22.3,
  'MSChange': -3.5,
  'Value_yr17': 289339,
  'Value_yr18': 175074},
 {'Country': 'Canada',
  'MS17': 14.9,
  'MS18': 13.6,
  'MSChange': -1.3,
  'Value_yr17': 167337,
  'Value_yr18': 107007},
 {'Country': 'India',
  'MS17': 15.5,
  'MS18': 11.5,
  'MSChange': -4.0,
  'Value_yr17': 174179,
  'Value_yr18': 90591},
 {'Country': 'Taiwan',
  'MS17': 6.0,
  'MS18': 8.2,
  'MSChange': 2.3,
  'Value_yr17': 67184,
  'Value_yr18': 64703},
 {'Country': 'Japan',
  'MS17': 3.0,
  'MS18': 5.0,
  'MSChange': 2.0,
  'Value_yr17': 33401,
  'Value_yr18': 39167}]

In [99]:
fi_summary = session.query(FIS.country,FIS.yr17,FED.yr18).group_by(FIS.country).all()
fi_summary_df = pd.DataFrame(fi_summary, columns=['country', 'yr17', 'yr18'])
fi_summary_df['yr17'] = fi_summary_df['yr17'].str.replace(',', '')
fi_summary_df['yr18'] = fi_summary_df['yr18'].str.replace(',', '')
fi_summary_df=fi_summary_df.dropna(how="any")
fi_summary_df["yr17"] = pd.to_numeric(fi_summary_df["yr17"])
fi_summary_df["yr18"] = pd.to_numeric(fi_summary_df["yr18"])

vi_summary = session.query(VIS.country,VIS.yr17,VIS.yr18).group_by(VIS.country).all()
vi_summary_df = pd.DataFrame(vi_summary, columns=['country', 'yr17', 'yr18'])
vi_summary_df['yr17'] = vi_summary_df['yr17'].str.replace(',', '')
vi_summary_df['yr18'] = vi_summary_df['yr18'].str.replace(',', '')
vi_summary_df=vi_summary_df.dropna(how="any")
vi_summary_df["yr17"] = pd.to_numeric(vi_summary_df["yr17"])
vi_summary_df["yr18"] = pd.to_numeric(vi_summary_df["yr18"])

isummary_frames = [vi_summary_df, fi_summary_df]
summary_import = pd.concat(isummary_frames,ignore_index=False)
summary_import["Total"] = summary_import["yr17"] + summary_import["yr18"]
summary_import = summary_import.sort_values(["Total"], ascending=False)
import_yr17sum=summary_import['yr17'].sum()
import_yr18sum=summary_import['yr18'].sum()

i_ms17=summary_import['yr17']/import_yr17sum*100
i_ms18=summary_import['yr18']/import_yr18sum*100
summary_import["MarketShare(2017)"]=i_ms17
summary_import["MarketShare(2018)"]=i_ms18
summary_import["ChangeInMarketShare"] = i_ms18-i_ms17
renamed_summary_import = summary_import.rename(columns={"country":"Country", "MS 2017":"MarketShare(2017)",
                                                        "MS 2018":"MarketShare(2018)", "Change in MS":"ChangeInMarketShare",
                                                        "yr17":"Value(2017)", "yr18":"Value(2018)"})
summary_import_list = []

renamed_summary_import = renamed_summary_import.reset_index(drop=True)
ms17_list = renamed_summary_import["MarketShare(2017)"].tolist()
ms18_list = renamed_summary_import["MarketShare(2018)"].tolist()
mschange_list = renamed_summary_import["ChangeInMarketShare"].tolist()
value_yr17 = renamed_summary_import["Value(2017)"].tolist()
value_yr18 = renamed_summary_import["Value(2018)"].tolist()
country_list = renamed_summary_import["Country"].tolist()

for i in range(len(country_list)):
    summary_report_dict = {}
    summary_report_dict["Country"] = country_list[i]
    summary_report_dict["MS17"] = round(ms17_list[i],1)
    summary_report_dict["MS18"] = round(ms18_list[i],1)
    summary_report_dict["MSChange"] = round(mschange_list[i],1)
    summary_report_dict["Value_yr17"] = value_yr17[i]
    summary_report_dict["Value_yr18"] = value_yr18[i]
    
    summary_import_list.append(summary_report_dict)
summary_import_list

[{'Country': 'Chile',
  'MS17': 31.5,
  'MS18': 12.1,
  'MSChange': -19.4,
  'Value_yr17': 104232,
  'Value_yr18': 175074},
 {'Country': 'New Zealand',
  'MS17': 23.4,
  'MS18': 12.1,
  'MSChange': -11.4,
  'Value_yr17': 77678,
  'Value_yr18': 175074},
 {'Country': 'Canada',
  'MS17': 11.3,
  'MS18': 12.1,
  'MSChange': 0.7,
  'Value_yr17': 37604,
  'Value_yr18': 175074},
 {'Country': 'Dominican Republic',
  'MS17': 9.8,
  'MS18': 12.1,
  'MSChange': 2.3,
  'Value_yr17': 32344,
  'Value_yr18': 175074},
 {'Country': 'Argentina',
  'MS17': 5.2,
  'MS18': 12.1,
  'MSChange': 6.9,
  'Value_yr17': 17139,
  'Value_yr18': 175074},
 {'Country': 'China',
  'MS17': 1.4,
  'MS18': 12.1,
  'MSChange': 10.7,
  'Value_yr17': 4506,
  'Value_yr18': 175074},
 {'Country': 'Uruguay',
  'MS17': 1.3,
  'MS18': 12.1,
  'MSChange': 10.8,
  'Value_yr17': 4270,
  'Value_yr18': 175074},
 {'Country': 'Greece',
  'MS17': 0.9,
  'MS18': 12.1,
  'MSChange': 11.2,
  'Value_yr17': 2976,
  'Value_yr18': 175074},
 {'Co

In [101]:
ExportDataVeggie = session.query(VED.country,VED.product,VED.yr17,VED.yr18).\
    order_by(VED.country.asc()).all()

ve_data_df = pd.DataFrame(ExportDataVeggie, columns=['country','product', 'yr17', 'yr18'])

ve_data_df['yr17'] = ve_data_df['yr17'].str.replace(',', '')
ve_data_df['yr18'] = ve_data_df['yr18'].str.replace(',', '')
ve_data_df = ve_data_df.dropna(how="any")
ve_data_df["yr17"] = pd.to_numeric(ve_data_df["yr17"])
ve_data_df["yr18"] = pd.to_numeric(ve_data_df["yr18"])
ve_data_df["average"] = (ve_data_df["yr17"] + ve_data_df["yr18"])/2
ve_data_df = ve_data_df.sort_values(["average"], ascending=False)
ve_data_result = ve_data_df.reset_index(drop=True)
ve_data_result = ve_data_result[:10]

country_list = ve_data_result["country"].tolist()
product_list = ve_data_result["product"].tolist()
avg_list = ve_data_result["average"].tolist()
veggie_yr17 = ve_data_result["yr17"].tolist()
veggie_yr18 = ve_data_result["yr18"].tolist()
vegexportdest_list = []

for i in range(len(country_list)):
    vegexportdest_dict = {}
    vegexportdest_dict["Country"] = country_list[i]
    vegexportdest_dict["Product"] = product_list[i]
    vegexportdest_dict["YR17"] = veggie_yr17[i]
    vegexportdest_dict["YR18"] = veggie_yr18[i]
    vegexportdest_dict["Average"] = avg_list[i]
    
    vegexportdest_list.append(vegexportdest_dict)
    
vegexportdest_list

[{'Country': 'Canada',
  'Product': 'potato',
  'YR17': 416004,
  'YR18': 141898,
  'Average': 278951.0},
 {'Country': 'Canada',
  'Product': 'onion',
  'YR17': 362415,
  'YR18': 80601,
  'Average': 221508.0},
 {'Country': 'Mexico',
  'Product': 'potato',
  'YR17': 237103,
  'YR18': 138089,
  'Average': 187596.0},
 {'Country': 'Canada',
  'Product': 'carrot',
  'YR17': 143754,
  'YR18': 154315,
  'Average': 149034.5},
 {'Country': 'Canada',
  'Product': 'sweet corn',
  'YR17': 141165,
  'YR18': 134033,
  'Average': 137599.0},
 {'Country': 'Canada',
  'Product': 'broccoli',
  'YR17': 114891,
  'YR18': 113061,
  'Average': 113976.0},
 {'Country': 'Mexico',
  'Product': 'onion',
  'YR17': 181056,
  'YR18': 16935,
  'Average': 98995.5},
 {'Country': 'Canada',
  'Product': 'pepper',
  'YR17': 99638,
  'YR18': 24523,
  'Average': 62080.5},
 {'Country': 'Japan',
  'Product': 'potato',
  'YR17': 74767,
  'YR18': 43376,
  'Average': 59071.5},
 {'Country': 'Taiwan',
  'Product': 'potato',
  'YR1

In [64]:
sel = [
    FED.country,
    FED.product,
    FED.yr17,
    FED.yr18,
]
ExportDataFruit = session.query(*sel).order_by(FED.country.desc()).all()
fe_data_df = pd.DataFrame(ExportDataFruit, columns=['country','product', 'yr17', 'yr18'])
fe_data_df["yr17"] = fe_data_df["yr17"].str.replace(",","")
fe_data_df["yr18"] = fe_data_df["yr18"].str.replace(",","")
fe_data_df = fe_data_df.dropna(how = 'any')
fe_data_df["yr17"] = pd.to_numeric(fe_data_df["yr17"])
fe_data_df["yr18"] = pd.to_numeric(fe_data_df["yr18"])
fe_data_df["average"] = (fe_data_df["yr17"] + fe_data_df["yr18"])/2
fe_data_df = fe_data_df.sort_values(["average"], ascending=False)
fe_data_result = fe_data_df.reset_index(drop=True)
fe_data_result = fe_data_result[:10]

country_list = fe_data_result["country"].tolist()
product_list = fe_data_result["product"].tolist()
avg_list = fe_data_result["average"].tolist()
fruit_yr17 = fe_data_result["yr17"].tolist()
fruit_yr18 = fe_data_result["yr18"].tolist()
fruitexportdest_list = []

for i in range(len(country_list)):
    fruitexportdest_dict = {}
    fruitexportdest_dict["Country"] = country_list[i]
    fruitexportdest_dict["Product"] = product_list[i]
    fruitexportdest_dict["YR17"] = fruit_yr17[i]
    fruitexportdest_dict["YR18"] = fruit_yr17[i]
    fruitexportdest_dict["Average"] = avg_list[i]
    
    fruitexportdest_list.append(fruitexportdest_dict)
    
fruitexportdest_list

[{'Country': 'Mexico',
  'Product': 'Apple',
  'YR17': 289339,
  'YR18': 289339,
  'Average': 232206.5},
 {'Country': 'Canada',
  'Product': 'Grapes',
  'YR17': 221500,
  'YR18': 221500,
  'Average': 221338.5},
 {'Country': 'Korea, South',
  'Product': 'Oranges',
  'YR17': 208080,
  'YR18': 208080,
  'Average': 187840.0},
 {'Country': 'Canada',
  'Product': 'Strawberries',
  'YR17': 304878,
  'YR18': 304878,
  'Average': 182910.5},
 {'Country': 'Canada',
  'Product': 'Apple',
  'YR17': 167337,
  'YR18': 167337,
  'Average': 137172.0},
 {'Country': 'India',
  'Product': 'Apple',
  'YR17': 174179,
  'YR18': 174179,
  'Average': 132385.0},
 {'Country': 'Canada',
  'Product': 'Oranges',
  'YR17': 131653,
  'YR18': 131653,
  'Average': 108682.0},
 {'Country': 'Mexico',
  'Product': 'Grapes',
  'YR17': 95209,
  'YR18': 95209,
  'Average': 95188.0},
 {'Country': 'Hong Kong',
  'Product': 'Oranges',
  'YR17': 86237,
  'YR18': 86237,
  'Average': 68175.5},
 {'Country': 'Mexico',
  'Product': 'P

In [85]:
    ImportDataFruit = session.query(FIS.country,FIS.product,FIS.yr17,VED.yr18).\
    order_by(FIS.country.asc()).all()

    fi_data_df = pd.DataFrame(ImportDataFruit, columns=['country','product', 'yr17', 'yr18'])
    fi_data_df['yr17'] = fi_data_df['yr17'].str.replace(',', '')
    fi_data_df['yr18'] = fi_data_df['yr18'].str.replace(',', '')
    fi_data_df = fi_data_df.dropna(how="any")
    fi_data_df["yr17"] = pd.to_numeric(fi_data_df["yr17"])
    fi_data_df["yr18"] = pd.to_numeric(fi_data_df["yr18"])
    fi_data_df["average"] = (fi_data_df["yr17"] + fi_data_df["yr18"])/2
    fi_data_df = fi_data_df.sort_values(["average"], ascending=False)
    fi_data_result = fi_data_df.reset_index(drop=True)
    fi_data_result = fi_data_result[:10]

    country_list = fi_data_result["country"].tolist()
    product_list = fi_data_result["product"].tolist()
    avg_list = fi_data_result["average"].tolist() 
    fruit_yr17 = fi_data_result["yr17"].tolist()
    fruit_yr18 = fi_data_result["yr18"].tolist()
    fruitimportsource_list = []

    for i in range(len(country_list)):
        fruitimportsource_dict = {}
        fruitimportsource_dict["Country"] = country_list[i]
        fruitimportsource_dict["Product"] = product_list[i]
        fruitimportsource_dict["YR17"] = fruit_yr17[i]
        fruitimportsource_dict["YR18"] = fruit_yr18[i]
        fruitimportsource_dict["Average"] = avg_list[i]
    
        fruitimportsource_list.append(fruitimportsource_dict)
    fruitimportsource_list

[{'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 154315,
  'Average': 1244483.0},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 141898,
  'Average': 1238274.5},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 138089,
  'Average': 1236370.0},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 134033,
  'Average': 1234342.0},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 113061,
  'Average': 1223856.0},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 80601,
  'Average': 1207626.0},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 45349,
  'Average': 1190000.0},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 43376,
  'Average': 1189013.5},
 {'Country': 'Mexico',
  'Product': 'Avocado',
  'YR17': 2334651,
  'YR18': 41729,
  'Average': 1188190.0},
 {'Country': 'Mexico',


In [102]:
sel = [
    FED.country,
    FED.product,
    FED.yr17,
    FED.yr18,
]
ExportDataFruit = session.query(*sel).group_by(FED.product).order_by(FED.country.desc()).all()
fe_data_df = pd.DataFrame(ExportDataFruit, columns=['country','product', 'yr17', 'yr18'])
fe_data_df['yr17'] = pd.to_numeric(fe_data_df['yr17'].str.replace(",",""))
fe_data_df['yr18'] = pd.to_numeric(fe_data_df['yr18'].str.replace(",",""))
fe_data_df = fe_data_df.dropna(how = 'any')
fe_data_df['Fruit_Export_Total'] = fe_data_df['yr17'] + fe_data_df['yr18']
fe_summary = fe_data_df.groupby(['product'])
fe_summary_df = pd.DataFrame({'Fruit_Export_Total': fe_summary['Fruit_Export_Total'].sum()})
fe_summary_df = fe_summary_df.sort_values(['Fruit_Export_Total'], ascending = False)
product_list = fe_summary_df.index.tolist()
fe_top10 = product_list[:10]
fe_top10_qty = fe_summary_df['Fruit_Export_Total'][:10].tolist()
fe_top10

['Apple',
 'Grapes',
 'Oranges',
 'Strawberries',
 'Pears',
 'Blueberries',
 'Avocado',
 'Grapefruit',
 'Citrus',
 'Tangerines']

In [74]:
ImportDataFruit = session.query(FIS.country,FIS.product,FIS.yr17,FIS.yr18).\
    order_by(FIS.country.asc()).all()

fi_data_df = pd.DataFrame(ImportDataFruit, columns=['country','product', 'yr17', 'yr18'])
fi_data_df['yr17'] = pd.to_numeric(fi_data_df['yr17'].str.replace(",",""))
fi_data_df['yr18'] = pd.to_numeric(fi_data_df['yr18'].str.replace(",",""))
fi_data_df = fi_data_df.dropna(how = 'any')
fi_data_df['Fruit_Import_Total'] = fi_data_df['yr17'] + fi_data_df['yr18']
fi_summary = fi_data_df.groupby(['product'])
fi_summary_df = pd.DataFrame({'Fruit_Import_Total': fi_summary['Fruit_Import_Total'].sum()})
fi_summary_df = fi_summary_df.sort_values(['Fruit_Import_Total'], ascending = False)
fi_top10_qty = fi_summary_df['Fruit_Import_Total'][:10].tolist()
import_list = fi_summary_df.index.tolist()
fi_top10 = import_list[:10]
fi_top10

['Avocado',
 'Grapes',
 'Blueberries',
 'Pineapples',
 'Strawberries',
 'Mango',
 'Limes',
 'Apple',
 'Oranges',
 'Kiwi']

In [103]:
    ExportDataVeggie = session.query(VED.country,VED.product,VED.yr17,VED.yr18).\
    order_by(VED.country.asc()).all()

    ve_data_df = pd.DataFrame(ExportDataVeggie, columns=['country','product', 'yr17', 'yr18'])

    ve_data_df['yr17'] = ve_data_df['yr17'].str.replace(',', '')
    ve_data_df['yr18'] = ve_data_df['yr18'].str.replace(',', '')
    ve_data_df = ve_data_df.dropna(how="any")
    ve_data_df["yr17"] = pd.to_numeric(ve_data_df["yr17"])
    ve_data_df["yr18"] = pd.to_numeric(ve_data_df["yr18"])
    ve_data_df["average"] = (ve_data_df["yr17"] + ve_data_df["yr18"])/2
    ve_data_df = ve_data_df.sort_values(["average"], ascending=False)
    ve_data_result = ve_data_df.reset_index(drop=True)
    
    ve_summary = ve_data_result.groupby(['product'])
    ve_summary_df = pd.DataFrame({'Veggie_Export': ve_summary['average'].sum()})
    ve_summary_df = ve_summary_df.sort_values(['Veggie_Export'], ascending = False)
    product_list = ve_summary_df.index.tolist()
    ve_top10 = product_list[:10]
    ve_top10_qty = ve_summary_df['Veggie_Export'][:10].tolist()
    ve_top10

['potato',
 'onion',
 'broccoli',
 'carrot',
 'sweet corn',
 'pepper',
 'beet',
 'green pea',
 'mushroom',
 'artichoke']

In [75]:
ImportDataVeggie = session.query(VIS.country,VIS.product,VIS.yr17,VED.yr18).\
    order_by(VIS.country.asc()).all()

vi_data_df = pd.DataFrame(ImportDataVeggie, columns=['country','product', 'yr17', 'yr18'])

vi_data_df['yr17'] = vi_data_df['yr17'].str.replace(',', '')
vi_data_df['yr18'] = vi_data_df['yr18'].str.replace(',', '')
vi_data_df = vi_data_df.dropna(how="any")
vi_data_df["yr17"] = pd.to_numeric(vi_data_df["yr17"])
vi_data_df["yr18"] = pd.to_numeric(vi_data_df["yr18"])
vi_data_df["average"] = (vi_data_df["yr17"] + vi_data_df["yr18"])/2
vi_data_df = vi_data_df.sort_values(["average"], ascending=False)
vi_data_result = vi_data_df.reset_index(drop=True)
vi_data_result = vi_data_result[:10]

country_list = vi_data_result["country"].tolist()
product_list = vi_data_result["product"].tolist()
avg_list = vi_data_result["average"].tolist()
veggie_yr17 = vi_data_result["yr17"].tolist()
veggie_yr18 = vi_data_result["yr18"].tolist()
vegimportsource_list = []

for i in range(len(country_list)):
    vegimportsource_dict = {}
    vegimportsource_dict["Country"] = country_list[i]
    vegimportsource_dict["Product"] = product_list[i]
    vegimportsource_dict["YR17"] = veggie_yr17[i]
    vegimportsource_dict["YR18"] = veggie_yr18[i]
    vegimportsource_dict["Average"] = avg_list[i]
    
    vegimportsource_list.append(vegimportsource_dict)
    
vegimportsource_list

[{'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 154315,
  'Average': 540279.5},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 141898,
  'Average': 534071.0},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 138089,
  'Average': 532166.5},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 134033,
  'Average': 530138.5},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 113061,
  'Average': 519652.5},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 80601,
  'Average': 503422.5},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 45349,
  'Average': 485796.5},
 {'Country': 'Canada',
  'Product': 'potato',
  'YR17': 926244,
  'YR18': 43376,
  'Average': 484810.0},
 {'Country': 'Mexico',
  'Product': 'pepper',
  'YR17': 814961,
  'YR18': 154315,
  'Average': 484638.0},
 {'Country': 'Canada',
  'Product': 'potato',
  '

In [61]:
ExportDataFruit = session.query(FED.country,FED.product,FED.yr17,FED.yr18).\
    order_by(FED.yr17.desc()).all()

fe_data_df = pd.DataFrame(ExportDataFruit, columns=['country','product', 'yr17', 'yr18'])

fe_data_df['yr17'] = fe_data_df['yr17'].str.replace(',', '')
fe_data_df['yr18'] = fe_data_df['yr18'].str.replace(',', '')
fe_data_df["yr17"] = pd.to_numeric(fe_data_df["yr17"])
fe_data_df["yr18"] = pd.to_numeric(fe_data_df["yr18"])
fe_data_df = fe_data_df.dropna(how="any")
fe_data_df.sort_values(['yr17', 'yr18'], ascending=False)

,country,product,yr17,yr18
64,Canada,Strawberries,304878.0,60943.0
84,Mexico,Apple,289339.0,175074.0
92,Canada,Grapes,221500.0,221177.0
96,"Korea, South",Oranges,208080.0,167600.0
121,India,Apple,174179.0,90591.0
124,Canada,Apple,167337.0,107007.0
131,Canada,Oranges,131653.0,85711.0
134,"Korea, South",Cherries,128427.0,2945.0
0,Mexico,Grapes,95209.0,95167.0
1,Canada,Blueberries,91238.0,2798.0


In [54]:
export_frames = [ve_data_df, fe_data_df]
all_export = pd.concat(export_frames)

In [55]:
all_export.sort_values(['yr17', 'yr18'], ascending=False).head()

,country,product,yr17,yr18
12,Canada,potato,416004,141898
10,Canada,onion,362415,80601
196,Canada,Strawberries,304878,60943
653,Mexico,Apple,289339,175074
32,Mexico,potato,237103,138089


In [56]:
all_export.count()

country    149
product    149
yr17       149
yr18       149
dtype: int64

In [57]:
all_export_yr18sum=all_export['yr18'].sum()

In [58]:
all_export_yr18sum

3336520

In [59]:
for i in all_export:
    eproduct_share=all_export["yr18"]/all_export_yr18sum*100

In [60]:
all_export["Share"]=eproduct_share

In [61]:
organized_all_export = all_export[["country","product","yr18","Share"]]
organized_all_import.head()

,country,product,yr18,Share
7,Canada,artichoke,2165,0.019858
8,Canada,beet,16551,0.151808
9,Canada,broccoli,9061,0.083109
10,Canada,carrot,206471,1.893786
12,Canada,mushroom,66446,0.609454


In [62]:
org_top10_export = organized_all_export.sort_values(['Share'], ascending=False).head(10)
org_top10_export

,country,product,yr18,Share
185,Canada,Grapes,221177,6.628973
653,Mexico,Apple,175074,5.247204
597,"Korea, South",Oranges,167600,5.023198
7,Canada,carrot,154315,4.625028
12,Canada,potato,141898,4.252874
32,Mexico,potato,138089,4.138713
13,Canada,sweet corn,134033,4.017150
6,Canada,broccoli,113061,3.388591
176,Canada,Apple,107007,3.207144
662,Mexico,Grapes,95167,2.852283


In [63]:
#Using .rename(columns={}) in order to rename columns
top10_export = org_top10_export.rename(columns={"country":"Country", "product":"Product","yr18":"$Value(2018)", "Share":"% Share"})
top10_export 

,Country,Product,$Value(2018),% Share
185,Canada,Grapes,221177,6.628973
653,Mexico,Apple,175074,5.247204
597,"Korea, South",Oranges,167600,5.023198
7,Canada,carrot,154315,4.625028
12,Canada,potato,141898,4.252874
32,Mexico,potato,138089,4.138713
13,Canada,sweet corn,134033,4.017150
6,Canada,broccoli,113061,3.388591
176,Canada,Apple,107007,3.207144
662,Mexico,Grapes,95167,2.852283


In [64]:
top10_export.reset_index(inplace=True)

In [65]:
# ########################<<<<<Top10_Export Data>>>>>######################################## 
Top10_Export= top10_export[["Country", "Product", "$Value(2018)", "% Share"]]
Top10_Export

,Country,Product,$Value(2018),% Share
0,Canada,Grapes,221177,6.628973
1,Mexico,Apple,175074,5.247204
2,"Korea, South",Oranges,167600,5.023198
3,Canada,carrot,154315,4.625028
4,Canada,potato,141898,4.252874
5,Mexico,potato,138089,4.138713
6,Canada,sweet corn,134033,4.017150
7,Canada,broccoli,113061,3.388591
8,Canada,Apple,107007,3.207144
9,Mexico,Grapes,95167,2.852283


In [66]:
fi_summary = session.query(FIS.country,FIS.yr17,FIS.yr18).\
    group_by(FIS.country).\
     order_by(FIS.yr18.desc()).all()
fi_summary_df = pd.DataFrame(fi_summary, columns=['country', 'yr17', 'yr18'])

In [67]:
fi_summary_df.count()

country    68
yr17       68
yr18       68
dtype: int64

In [68]:
fi_summary_df['yr17'] = fi_summary_df['yr17'].str.replace(',', '')

In [69]:
fi_summary_df['yr18'] = fi_summary_df['yr18'].str.replace(',', '')

In [70]:
fi_summary_df.count()

country    68
yr17        8
yr18        8
dtype: int64

In [71]:
fi_summary_df.fillna(0).head()

,country,yr17,yr18
0,Chile,104232,90268
1,New Zealand,77678,70271
2,China,4506,5665
3,Uruguay,4270,4159
4,Dominican Republic,32344,36660


In [72]:
fi_summary_df=fi_summary_df.dropna(how="any")

In [73]:
fi_summary_df["yr17"] = pd.to_numeric(fi_summary_df["yr17"])
fi_summary_df["yr18"] = pd.to_numeric(fi_summary_df["yr18"])

In [74]:
fi_summary_df.count()

country    8
yr17       8
yr18       8
dtype: int64

In [75]:
vi_summary = session.query(VIS.country,VIS.yr17,VIS.yr18).\
    group_by(VIS.country).\
     order_by(VIS.yr18.desc()).all()
vi_summary_df = pd.DataFrame(vi_summary, columns=['country', 'yr17', 'yr18'])

In [76]:
vi_summary_df.count()

country    38
yr17       38
yr18       38
dtype: int64

In [77]:
vi_summary_df['yr17'] = vi_summary_df['yr17'].str.replace(',', '')

In [78]:
vi_summary_df['yr18'] = vi_summary_df['yr18'].str.replace(',', '')

In [79]:
vi_summary_df.fillna(0).head()

,country,yr17,yr18
0,Israel,42295,42982
1,China,6032,4753
2,Canada,2271,2165
3,Mexico,0,0
4,Thailand,0,0


In [80]:
vi_summary_df=vi_summary_df.dropna(how="any")

In [81]:
vi_summary_df["yr17"] = pd.to_numeric(vi_summary_df["yr17"])
vi_summary_df["yr18"] = pd.to_numeric(vi_summary_df["yr18"])

In [82]:
vi_summary_df.count()

country    3
yr17       3
yr18       3
dtype: int64

In [83]:
fe_summary = session.query(FED.country,FED.yr17,FED.yr18).\
    group_by(FED.country).\
     order_by(FED.yr18.desc()).all()
fe_summary_df = pd.DataFrame(fe_summary, columns=['country', 'yr17', 'yr18'])

In [84]:
fe_summary_df['yr17'] = fe_summary_df['yr17'].str.replace(',', '')

In [85]:
fe_summary_df['yr18'] = fe_summary_df['yr18'].str.replace(',', '')

In [86]:
fe_summary_df.fillna(0).head()

,country,yr17,yr18
0,India,174179,90591
1,Costa Rica,11646,8941
2,Malaysia,11073,8034
3,Honduras,9196,7294
4,Taiwan,67184,64703


In [87]:
fe_summary_df=fe_summary_df.dropna(how="any")

In [88]:
fe_summary_df["yr17"] = pd.to_numeric(fe_summary_df["yr17"])
fe_summary_df["yr18"] = pd.to_numeric(fe_summary_df["yr18"])

In [89]:
fe_summary_df.count()

country    36
yr17       36
yr18       36
dtype: int64

In [90]:
ve_summary = session.query(VED.country,VED.yr17,VED.yr18).\
    group_by(VED.country).\
     order_by(VED.yr18.desc()).all()
ve_summary_df = pd.DataFrame(ve_summary, columns=['country', 'yr17', 'yr18'])

In [91]:
ve_summary_df['yr17'] = ve_summary_df['yr17'].str.replace(',', '')

In [92]:
ve_summary_df['yr18'] = ve_summary_df['yr18'].str.replace(',', '')

In [93]:
ve_summary_df.fillna(0).head()

,country,yr17,yr18
0,United Kingdom,1943,5849
1,Taiwan,9706,4807
2,Japan,33401,39167
3,Canada,2678,2990
4,"Korea, South",1432,1343


In [94]:
ve_summary_df=ve_summary_df.dropna(how="any")

In [95]:
ve_summary_df["yr17"] = pd.to_numeric(ve_summary_df["yr17"])
ve_summary_df["yr18"] = pd.to_numeric(ve_summary_df["yr18"])

In [96]:
ve_summary_df.count()

country    5
yr17       5
yr18       5
dtype: int64

In [97]:
isummary_frames = [vi_summary_df, fi_summary_df]
summary_import = pd.concat(isummary_frames, ignore_index=True)

# fe_summary = session.query(FED.country,FED.yr17,FED.yr18).\
#     group_by(FED.country).\
#      order_by(FED.yr18.desc()).all()

In [98]:
summary_import.sort_values(['yr17', 'yr18'], ascending=False).head()

,country,yr17,yr18
3,Chile,104232,90268
4,New Zealand,77678,70271
0,Israel,42295,42982
9,Canada,37604,24737
7,Dominican Republic,32344,36660


In [99]:
import_yr17sum=summary_import['yr17'].sum()

In [100]:
import_yr17sum

331347

In [101]:
import_yr18sum=summary_import['yr18'].sum()

In [102]:
import_yr18sum

295615

In [103]:
summary_import.count()

country    11
yr17       11
yr18       11
dtype: int64

In [104]:
for i in summary_import:
    ms17=summary_import['yr17']/import_yr17sum*100


In [105]:
for i in summary_import:
    ms18=summary_import['yr18']/import_yr18sum*100


In [106]:
# thousands_of_dollars = data_file_pd["Amount"]/1000
# data_file_pd["Thousands of Dollars"] = thousands_of_dollars

# data_file_pd.head() 

In [107]:
summary_import["MS 2017"]=ms17

In [108]:
summary_import["MS 2018"]=ms18

In [109]:
change = ms18-ms17

In [110]:
summary_import["Change in MS"]=change

In [111]:
summary_import.head()

,country,yr17,yr18,MS 2017,MS 2018,Change in MS
0,Israel,42295,42982,12.764564,14.539858,1.775294
1,China,6032,4753,1.820448,1.607835,-0.212614
2,Canada,2271,2165,0.685384,0.732371,0.046987
3,Chile,104232,90268,31.457053,30.535663,-0.921390
4,New Zealand,77678,70271,23.443097,23.771121,0.328024


In [112]:
organized_summary_import = summary_import[["country","MS 2017","MS 2018","Change in MS", "yr17", "yr18"]]
organized_summary_import.head()

,country,MS 2017,MS 2018,Change in MS,yr17,yr18
0,Israel,12.764564,14.539858,1.775294,42295,42982
1,China,1.820448,1.607835,-0.212614,6032,4753
2,Canada,0.685384,0.732371,0.046987,2271,2165
3,Chile,31.457053,30.535663,-0.921390,104232,90268
4,New Zealand,23.443097,23.771121,0.328024,77678,70271


In [113]:
#Using .rename(columns={}) in order to rename columns
summary_import = organized_summary_import.rename(columns={"country":"Country", "MS 2017":"Market Share(2017)","MS 2018":"Market Share(2018)", "Change in MS":"Change In Market Share","yr17":"$Value(2017)", "yr18":"$Value(2018)"})
summary_import.head()

,Country,Market Share(2017),Market Share(2018),Change In Market Share,$Value(2017),$Value(2018)
0,Israel,12.764564,14.539858,1.775294,42295,42982
1,China,1.820448,1.607835,-0.212614,6032,4753
2,Canada,0.685384,0.732371,0.046987,2271,2165
3,Chile,31.457053,30.535663,-0.921390,104232,90268
4,New Zealand,23.443097,23.771121,0.328024,77678,70271


In [114]:
summary_import=summary_import.sort_values(['Market Share(2018)'], ascending=False).head()
summary_import

,Country,Market Share(2017),Market Share(2018),Change In Market Share,$Value(2017),$Value(2018)
3,Chile,31.457053,30.535663,-0.921390,104232,90268
4,New Zealand,23.443097,23.771121,0.328024,77678,70271
0,Israel,12.764564,14.539858,1.775294,42295,42982
7,Dominican Republic,9.761368,12.401265,2.639897,32344,36660
9,Canada,11.348828,8.367979,-2.980849,37604,24737


In [115]:
summary_import.reset_index(inplace=True)

In [116]:
# ###################################<<<<<<<Summary_Import Data>>>>>>#################################################
Summary_Import = summary_import[["Country", "Market Share(2017)", "Market Share(2018)", "Change In Market Share", "$Value(2017)", "$Value(2018)"]]
Summary_Import

,Country,Market Share(2017),Market Share(2018),Change In Market Share,$Value(2017),$Value(2018)
0,Chile,31.457053,30.535663,-0.921390,104232,90268
1,New Zealand,23.443097,23.771121,0.328024,77678,70271
2,Israel,12.764564,14.539858,1.775294,42295,42982
3,Dominican Republic,9.761368,12.401265,2.639897,32344,36660
4,Canada,11.348828,8.367979,-2.980849,37604,24737


In [117]:
esummary_frames = [ve_summary_df, fe_summary_df]
summary_export = pd.concat(esummary_frames,ignore_index=False)


In [118]:
summary_export.sort_values(['yr17', 'yr18'], ascending=False).head()

,country,yr17,yr18
21,Mexico,289339,175074
0,India,174179,90591
28,Canada,167337,107007
4,Taiwan,67184,64703
11,Vietnam,41656,37473


In [119]:
summary_export.count()

country    41
yr17       41
yr18       41
dtype: int64

In [120]:
export_yr17sum=summary_export['yr17'].sum()

In [121]:
export_yr17sum

1124491

In [122]:
export_yr18sum=summary_export['yr18'].sum()

In [123]:
export_yr18sum

786676

In [124]:
# Reorganizing the columns using double brackets
# organized_df = training_data[["Name","Trainer","Weight","Membership(Days)"]]
# organized_df.head()

In [125]:
# # Using .rename(columns={}) in order to rename columns
# renamed_df = organized_df.rename(columns={"Membership(Days)":"Membership in Days", "Weight":"Weight in Pounds"})
# renamed_df.head()

In [126]:
for i in summary_export:
    e_ms17=summary_export['yr17']/export_yr17sum*100


In [127]:
for i in summary_export:
    e_ms18=summary_export['yr18']/export_yr18sum*100

In [128]:
summary_export["Market Share(2017)"]=e_ms17

In [129]:
summary_export["Market Share(2018)"]=e_ms18

In [130]:
summary_export.head()

,country,yr17,yr18,Market Share(2017),Market Share(2018)
0,United Kingdom,1943,5849,0.172789,0.743508
1,Taiwan,9706,4807,0.863146,0.611052
2,Japan,33401,39167,2.970322,4.978797
3,Canada,2678,2990,0.238152,0.380080
4,"Korea, South",1432,1343,0.127347,0.170718


In [131]:
e_change = e_ms18-e_ms17

In [132]:
summary_export["Change in Market Share"]=e_change

In [133]:
renamed_summary_export = summary_export.rename(columns={"country":"Country", "MS 2017":"Market Share(2017)","MS 2018":"Market Share(2018)", "Change in MS":"Change In Market Share","yr17":"$Value(2017)", "yr18":"$Value(2018)"})
renamed_summary_export.head()

,Country,$Value(2017),$Value(2018),Market Share(2017),Market Share(2018),Change in Market Share
0,United Kingdom,1943,5849,0.172789,0.743508,0.570719
1,Taiwan,9706,4807,0.863146,0.611052,-0.252094
2,Japan,33401,39167,2.970322,4.978797,2.008475
3,Canada,2678,2990,0.238152,0.380080,0.141928
4,"Korea, South",1432,1343,0.127347,0.170718,0.043372


In [134]:
summary_export = renamed_summary_export[["Country","Market Share(2017)","Market Share(2018)","Change in Market Share", "$Value(2017)", "$Value(2018)"]]
summary_export.head()

,Country,Market Share(2017),Market Share(2018),Change in Market Share,$Value(2017),$Value(2018)
0,United Kingdom,0.172789,0.743508,0.570719,1943,5849
1,Taiwan,0.863146,0.611052,-0.252094,9706,4807
2,Japan,2.970322,4.978797,2.008475,33401,39167
3,Canada,0.238152,0.380080,0.141928,2678,2990
4,"Korea, South",0.127347,0.170718,0.043372,1432,1343


In [135]:
summary_export=summary_export.sort_values(['Market Share(2018)'], ascending=False).head()
summary_export

,Country,Market Share(2017),Market Share(2018),Change in Market Share,$Value(2017),$Value(2018)
21,Mexico,25.730664,22.254905,-3.475758,289339,175074
28,Canada,14.881133,13.602423,-1.278710,167337,107007
0,India,15.489586,11.515668,-3.973917,174179,90591
4,Taiwan,5.974614,8.224860,2.250246,67184,64703
2,Japan,2.970322,4.978797,2.008475,33401,39167


In [136]:
summary_export.reset_index(inplace=True)

In [137]:
# #######################################<<<<<Summary_Export Data>>>>>>###########################################
Summary_Export = summary_export[["Country", "Market Share(2017)", "Market Share(2018)", "Change in Market Share", "$Value(2017)", "$Value(2018)"]]
Summary_Export

,Country,Market Share(2017),Market Share(2018),Change in Market Share,$Value(2017),$Value(2018)
0,Mexico,25.730664,22.254905,-3.475758,289339,175074
1,Canada,14.881133,13.602423,-1.278710,167337,107007
2,India,15.489586,11.515668,-3.973917,174179,90591
3,Taiwan,5.974614,8.224860,2.250246,67184,64703
4,Japan,2.970322,4.978797,2.008475,33401,39167
